In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [203]:
train_data = pd.read_csv(r"/kaggle/input/titanic/train.csv")
test_data = pd.read_csv(r"/kaggle/input/titanic/test.csv")

In [204]:
len(test_data)

418

In [35]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [37]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [38]:
#lets handle null values of Age
train_data["Age"] = train_data["Age"].fillna(train_data.groupby("Pclass")["Age"]
                                             .transform("mean"))

#I filled the null values of age with mean age value grouped by Pclass

In [39]:
train_data["Age"].isna().sum()

0

In [40]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [41]:
train_data["Deck"] = train_data["Cabin"].str[0]

In [42]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN


In [43]:
train_data.drop("Cabin", axis=1, inplace=True)

In [44]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN


In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [47]:
#lets handle missing values in Deck column by predicting corresponding 
#values from Fare column

train_data.fillna("U", inplace=True)
# Separate the rows with and without missing Deck values
known_deck = train_data[train_data['Deck'] != 'U']
unknown_deck = train_data[train_data['Deck'] == 'U']

# Features and target variable
X = known_deck[['Fare']]  # Ensure X is a DataFrame
y = known_deck['Deck']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Predict the missing Deck values
train_data.loc[train_data['Deck'] == 'U', 'Deck'] = model.predict(unknown_deck[['Fare']])

In [48]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,F
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,E
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,E


In [49]:
train_data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Deck           0
dtype: int64

In [72]:
#lets train our model
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [57]:
encoder = LabelEncoder()

In [52]:
numerical_cols = train_data.select_dtypes(exclude="object").columns
cat_cols = train_data.select_dtypes(include="object").columns
print(numerical_cols)
print(cat_cols)

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Index(['Name', 'Sex', 'Ticket', 'Embarked', 'Deck'], dtype='object')


In [58]:
train_data["Sex_Encoder"] = encoder.fit_transform(train_data["Sex"])
train_data["Embarked_Encoder"] = encoder.fit_transform(train_data["Embarked"])
train_data["Deck_Encoder"] = encoder.fit_transform(train_data["Deck"])

In [59]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck,Sex_Encoder,Embarked_Encoder,Deck_Encoder
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,F,1,2,5
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C,0,0,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,E,0,2,4
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C,0,2,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,E,1,2,4


In [62]:
train_data.drop(["Name","Sex","Ticket","Embarked","Deck"], axis=1, inplace=True)
train_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_Encoder,Embarked_Encoder,Deck_Encoder
0,1,0,3,22.0,1,0,7.2500,1,2,5
1,2,1,1,38.0,1,0,71.2833,0,0,2
2,3,1,3,26.0,0,0,7.9250,0,2,4
3,4,1,1,35.0,1,0,53.1000,0,2,2
4,5,0,3,35.0,0,0,8.0500,1,2,4


In [60]:
from sklearn.preprocessing import StandardScaler

In [63]:
scaler = StandardScaler()

scaled_features = scaler.fit_transform(train_data.drop(["PassengerId","Survived"], axis=1))

In [64]:
scaled_features

array([[ 0.82737724, -0.55236029,  0.43279337, ...,  0.73769513,
         0.58111394,  0.83807807],
       [-1.56610693,  0.65947522,  0.43279337, ..., -1.35557354,
        -1.93846038, -0.95120522],
       [ 0.82737724, -0.24940141, -0.4745452 , ..., -1.35557354,
         0.58111394,  0.24165031],
       ...,
       [ 0.82737724, -0.31449064,  0.43279337, ..., -1.35557354,
         0.58111394,  0.83807807],
       [-1.56610693, -0.24940141, -0.4745452 , ...,  0.73769513,
        -1.93846038, -0.95120522],
       [ 0.82737724,  0.20503691, -0.4745452 , ...,  0.73769513,
        -0.67867322,  0.83807807]])

In [65]:
X = scaled_features
y = train_data["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
classifier = LogisticRegression(max_iter=1000)

In [74]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [75]:
prediction = classifier.predict(X_test)

In [76]:
accuracy = accuracy_score(y_test, prediction)
cm = confusion_matrix(y_test, prediction)
cr = classification_report(y_test, prediction)

print(accuracy)
print(cm)
print(cr)

0.8156424581005587
[[91 14]
 [19 55]]
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       105
           1       0.80      0.74      0.77        74

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.81       179
weighted avg       0.81      0.82      0.81       179



In [77]:
#lets train our model on test dataset

In [205]:
len(test_data)

418

In [206]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [207]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [208]:
#test_data.dropna(subset=["Fare"], inplace=True)
test_data["Fare"].fillna(test_data["Fare"].median(), inplace=True)

/tmp/ipykernel_36/2601802368.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data["Fare"].fillna(test_data["Fare"].median(), inplace=True)


In [209]:
test_data["Age"] = test_data["Age"].fillna(test_data.groupby("Pclass")["Age"].transform("mean"))

In [210]:
test_data["Deck"] = test_data["Cabin"].str[0]
#test_data.head()

In [211]:
test_data.drop("Cabin", axis=1, inplace=True)

In [212]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
Deck           327
dtype: int64

In [213]:
#lets handle missing values in Deck column by predicting corresponding 
#values from Fare column

test_data["Deck"].fillna("U", inplace=True)
# Separate the rows with and without missing Deck values
known_deck = test_data[test_data['Deck'] != 'U']
unknown_deck = test_data[test_data['Deck'] == 'U']

# Features and target variable
X = known_deck[['Fare']]  # Ensure X is a DataFrame
y = known_deck['Deck']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Predict the missing Deck values
test_data.loc[test_data['Deck'] == 'U', 'Deck'] = model.predict(unknown_deck[['Fare']])

/tmp/ipykernel_36/1069524151.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data["Deck"].fillna("U", inplace=True)


In [214]:
test_data.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Deck           0
dtype: int64

In [215]:
test_data.drop(["Name","Ticket"], axis=1, inplace=True)

In [216]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,892,3,male,34.5,0,0,7.8292,Q,F
1,893,3,female,47.0,1,0,7.0000,S,F
2,894,2,male,62.0,0,0,9.6875,Q,F
3,895,3,male,27.0,0,0,8.6625,S,F
4,896,3,female,22.0,1,1,12.2875,S,F


In [217]:
test_data["Sex_Encoder"] = encoder.fit_transform(test_data["Sex"])
test_data["Embarked_Encoder"] = encoder.fit_transform(test_data["Embarked"])
test_data["Deck_Encoder"] = encoder.fit_transform(test_data["Deck"])

In [218]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Sex_Encoder,Embarked_Encoder,Deck_Encoder
0,892,3,male,34.5,0,0,7.8292,Q,F,1,1,5
1,893,3,female,47.0,1,0,7.0000,S,F,0,2,5
2,894,2,male,62.0,0,0,9.6875,Q,F,1,1,5
3,895,3,male,27.0,0,0,8.6625,S,F,1,2,5
4,896,3,female,22.0,1,1,12.2875,S,F,0,2,5


In [219]:
scaled_features_test = scaler.fit_transform(test_data.drop(["PassengerId","Sex","Embarked","Deck"], axis=1))

In [220]:
prediction_test = classifier.predict(scaled_features_test)

In [221]:
test_data["Survived"] = prediction_test

In [222]:
test_data.head(20)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Sex_Encoder,Embarked_Encoder,Deck_Encoder,Survived
0,892,3,male,34.500000,0,0,7.8292,Q,F,1,1,5,0
1,893,3,female,47.000000,1,0,7.0000,S,F,0,2,5,0
2,894,2,male,62.000000,0,0,9.6875,Q,F,1,1,5,0
3,895,3,male,27.000000,0,0,8.6625,S,F,1,2,5,0
4,896,3,female,22.000000,1,1,12.2875,S,F,0,2,5,1
5,897,3,male,14.000000,0,0,9.2250,S,F,1,2,5,0
6,898,3,female,30.000000,0,0,7.6292,Q,F,0,1,5,1
7,899,2,male,26.000000,1,1,29.0000,S,C,1,2,2,0
8,900,3,female,18.000000,0,0,7.2292,C,F,0,0,5,1
9,901,3,male,21.000000,2,0,24.1500,S,C,1,2,2,0


In [223]:
len(test_data)

418

In [224]:
final_test = test_data[["PassengerId","Survived"]].copy()

In [225]:
len(final_test)

418

In [227]:
final_test.to_csv("submission.csv", index=False)

In [228]:
from IPython.display import FileLink

In [229]:
FileLink('submission.csv')

/kaggle/working/submission.csv